In [1]:
import os

In [2]:
%pwd

'/Users/mani/Desktop/mlops/datascienceproject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/mani/Desktop/mlops/datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:  ## we need to mention all the variables which mentioned in config.yaml & atlast we need to assign all these values in pipelene
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [7]:
## whenver this configurationmanager calls then whaterver mwntioned in config, schema, params file needs to be call here
## so using config_filepath we are getting all info in those 3 files totally

## This class manages configurations for various stages of the ML pipeline
class ConfigurationManager: 

    ## Initializes the ConfigurationManager instance, loading configurations from specified files
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,  ## Path to main config file (default defined by CONFIG_FILE_PATH)
                 params_filepath=PARAMS_FILE_PATH,  ## Path to parameters file for model parameters (default defined by PARAMS_FILE_PATH)
                 schema_filepath=SCHEMA_FILE_PATH): ## Path to schema file for data validation (default defined by SCHEMA_FILE_PATH)

        # Reads YAML content from the provided config file path
        self.config = read_yaml(config_filepath)  
        
        # Reads YAML content from the parameters file for model hyperparameters
        self.params = read_yaml(params_filepath)
        
        # Reads YAML content from the schema file for data validation schema
        self.schema = read_yaml(schema_filepath)

        ## Creates an artifacts folder (or any other specified root directory) to store all generated files
        create_directories([self.config.artifacts_root]) 

    ## we ned to intiate data injection so we need all config info here, thats y creatingthis function get data injection config
    ## Method to retrieve data ingestion configurations; called whenever we need settings for the data ingestion step
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        # Extracts the 'data_ingestion' section from the main configuration file
        config = self.config.data_ingestion

        # Ensures the root directory for data ingestion is created and ready to store ingested data
        create_directories([config.root_dir])

        # Initializes and returns a DataIngestionConfig object with specific configurations for data ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,  ## Directory to store ingested data files
            source_URL=config.source_URL,  ## URL for downloading data
            local_data_file=config.local_data_file,  ## Local path to save the downloaded data file
            unzip_dir=config.unzip_dir  ## Directory to extract the downloaded file

        )
        return data_ingestion_config  ##this data injecton config will parse to my data injection pipeline Returns the configured DataIngestionConfig instance


In [8]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [9]:
## Data Ingestion component responsible for handling data downloading and extraction
class DataIngestion:
    
    ## Initializes the DataIngestion instance with a given configuration
    def __init__(self, config: DataIngestionConfig):
        
        # Sets the configuration for data ingestion, provided by a DataIngestionConfig object
        self.config = config

    ## Downloads the file from a URL if it does not already exist locally
    def download_file(self):
        
        # Checks if the file does not already exist at the specified path
        if not os.path.exists(self.config.local_data_file):
            
            # Downloads the file from the source URL to the local file path
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,  ## URL to download the file from
                filename=self.config.local_data_file  ## Local file path to save the downloaded file
            )
            
            # Logs the download details with filename and headers
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        
        # If the file exists, logs a message that it already exists
        else:
            logger.info(f"File already exists")

    ## Extracts the downloaded zip file into the specified directory
    def extract_zip_file(self):
        """
        Extracts the zip file contents into the directory specified in the configuration
        Function returns None
        """
        
        # Sets the path to unzip files to, as specified in the configuration
        unzip_path = self.config.unzip_dir
        
        # Ensures the target directory for unzipping exists; if not, creates it
        os.makedirs(unzip_path, exist_ok=True)
        
        # Opens the downloaded zip file in read mode to extract its contents
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            
            # Extracts all files in the zip to the specified directory
            zip_ref.extractall(unzip_path)


In [ ]:
# Start of a try-except block to handle any exceptions during the data ingestion process
## finally we are merging all downloading fliles , unzip file 
try:
    
    # Initializes a ConfigurationManager instance to load configuration files
    config = ConfigurationManager()
    
    # Retrieves the data ingestion configuration using the ConfigurationManager instance
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Initializes a DataIngestion instance with the retrieved data ingestion configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Calls the method to download the file from the specified URL if it is not already present locally
    data_ingestion.download_file()
    
    # Calls the method to extract the contents of the downloaded zip file to the specified directory
    data_ingestion.extract_zip_file()

# Catches any exceptions that occur within the try block
except Exception as e:
    
    # Re-raises the caught exception to propagate the error, so it can be handled or logged by higher-level code
    raise e


[2024-11-05 16:35:42,026: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-05 16:35:42,028: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-05 16:35:42,030: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-05 16:35:42,031: INFO: common: created directory at: artifacts]
[2024-11-05 16:35:42,031: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-05 16:35:42,878: INFO: 2284664551: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C4FE:38F039:1D681E:245137:6729FC06
Accept-Ranges: bytes
Date: Tu